In [1]:
pip install pydrive

     |████████████████████████████████| 993kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


IMPORTING THE NECESSARY LIBRARIES

In [12]:
# used to avoid python version clashes 
from __future__ import print_function
# IPython is a interface in python to display things
from IPython import display
# used to draw graphs and plots 
from matplotlib import pyplot as plt
# used for colormap
from matplotlib import cm
# used for subplot so that its easy to align the graph in a grid
from matplotlib import gridspec
# pandas is a python library used to create dataframe for tensorflow
import pandas as pd
# numpy is used for mathematical operations
import numpy as np
# open source AI library to make models
import tensorflow as tf
# used to find the MSE (Mean Squared Error)
from sklearn import metrics
# used to create tensors from the dataframe 
from tensorflow.python.data import Dataset
# used for mathematical operations
import math
# used to get the dataset which is stored on google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# used to display errors as ERRORS other option is to display it as WARNING
tf.logging.set_verbosity(tf.logging.ERROR)
# setting the maximum number of rows to display as 10
pd.options.display.max_rows = 10
# setting the float format as .1f
pd.options.display.float_format = '{:.1f}'.format
# authentication user account on google colab
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=11BCnuBHvALPEVnJ-9F9KJtFFBVyjpxJQ'
waste, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Iris.csv')  
# reading the dataset from google drive
iris_dataframe = pd.read_csv('Iris.csv')
# displaying the result
display.display(iris_dataframe)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


THERE IS NO WAY A LEARNING ALGORITHM CAN ACTUALLY LEARN FROM A DATASET THAT CONTAINS CATEGORICAL DATA

OPTIONS ARE EITHER WE CAN MANUALLY COVERT THE CATEGORICAL DATA INTO NUMERICAL DATA OR WE CAN USE ONE HOT ENCODING


In [0]:
# mapping the species so that we can conver the categorical data into numeric data
iris_dataframe["Species"] = iris_dataframe["Species"].map({"Iris-setosa":0,"Iris-virginica":1,"Iris-versicolor":2})
iris_dataframe = iris_dataframe.reindex(np.random.permutation(iris_dataframe.index))

In [47]:
# displaying the dataset
iris_dataframe.tail(50)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
97,98,6.2,2.9,4.3,1.3,2
74,75,6.4,2.9,4.3,1.3,2
79,80,5.7,2.6,3.5,1.0,2
23,24,5.1,3.3,1.7,0.5,0
35,36,5.0,3.2,1.2,0.2,0
...,...,...,...,...,...,...
139,140,6.9,3.1,5.4,2.1,1
68,69,6.2,2.2,4.5,1.5,2
145,146,6.7,3.0,5.2,2.3,1
9,10,4.9,3.1,1.5,0.1,0


SPLITING THE DATASET INTO TRAINING AND VALIDATION DATA

In [0]:
# function to get the features from the dataset
def features(iris_dataframe):
  # taking only the necessary features
  selected_features = iris_dataframe[
      [
          "SepalLengthCm",
          "SepalWidthCm",
          "PetalLengthCm",
          "PetalWidthCm"
      ]
  ]
  # storing them in a variable
  processed_features = selected_features.copy()
  # returning the result
  return processed_features

# function to get the label from the dataset
def labels(iris_dataframe):
  # creating a dataframe to store the label
  processed_label = pd.DataFrame()
  # assigning the label value to the variable
  processed_label["Species"] = (iris_dataframe["Species"])
  # returning the result
  return processed_label

In [19]:
# training data
training_examples = features(iris_dataframe.head(100))
display.display(training_examples)
training_targets = labels(iris_dataframe.head(100))
display.display(training_targets)

# validation data
validation_examples = features(iris_dataframe.tail(50))
display.display(validation_examples)
validation_targets = labels(iris_dataframe.tail(50))
display.display(validation_targets)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
15,5.7,4.4,1.5,0.4
110,6.5,3.2,5.1,2.0
73,6.1,2.8,4.7,1.2
53,5.5,2.3,4.0,1.3
82,5.8,2.7,3.9,1.2
...,...,...,...,...
20,5.4,3.4,1.7,0.2
31,5.4,3.4,1.5,0.4
40,5.0,3.5,1.3,0.3
65,6.7,3.1,4.4,1.4


,Species
15,0
110,1
73,2
53,2
82,2
...,...
20,0
31,0
40,0
65,2


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
97,6.2,2.9,4.3,1.3
74,6.4,2.9,4.3,1.3
79,5.7,2.6,3.5,1.0
23,5.1,3.3,1.7,0.5
35,5.0,3.2,1.2,0.2
...,...,...,...,...
139,6.9,3.1,5.4,2.1
68,6.2,2.2,4.5,1.5
145,6.7,3.0,5.2,2.3
9,4.9,3.1,1.5,0.1


,Species
97,2
74,2
79,2
23,0
35,0
...,...
139,1
68,2
145,1
9,0


In [0]:
def input_fn(features,labels,num_epochs=None,batch_size=1,shuffle=True):
  
  features = {key:np.array(value) for key,value in dict(features).items()}
  df = Dataset.from_tensor_slices((features,labels))
  df = df.batch(batch_size).repeat(num_epochs)
  
  if(shuffle==True):
    df = df.shuffle(1000)
  
  features,labels = df.make_one_shot_iterator().get_next()
  return features,labels

In [0]:
def create_feature_column(iris_dataframe):
  
  return set([tf.feature_column.numeric_column(features) for features in iris_dataframe])

In [49]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=create_feature_column(training_examples),
    hidden_units=[10, 10],
    n_classes=3)

classifier.train(
    input_fn=lambda:input_fn(training_examples,training_targets,batch_size=100),
    steps=100)
eval_result = classifier.evaluate(
    input_fn=lambda:input_fn(training_examples,training_targets,batch_size=10),steps=100)

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLengthCm': [5.1, 5.9, 6.9],
    'SepalWidthCm': [3.3, 3.0, 3.1],
    'PetalLengthCm': [1.7, 4.2, 5.4],
    'PetalWidthCm': [0.5, 1.5, 2.1],
}

def eval_input_fn(features, labels, batch_size):
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

predictions = classifier.predict(
    input_fn = lambda : eval_input_fn(features = predict_x, labels = None, batch_size = 100)
)

  
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

expected = ['Setosa', 'Versicolor', 'Virginica']

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id], 100 * probability, expec))
   


Test set accuracy: 0.992


Prediction is "Setosa" (96.7%), expected "Setosa"

Prediction is "Virginica" (85.3%), expected "Versicolor"

Prediction is "Versicolor" (66.2%), expected "Virginica"
